# Neural Network

## Imports

In [51]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils

## Activatie functies, mean squared error en hun afgeleides

In [52]:
def sig(x):
     return 1/(1 + np.exp(-x))

def relu(x):
    return max(0.0, x)

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def der_relu(x):
    return relu(x)

def der_sig(x):
    return sig(x) * (1.0 - sig(x))

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def der_mse(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

## Layer

In [53]:
class Layer():
    # het initialiseren van de laag
    def __init__(self, input_data, output_data, activation, der_activation):
        self.input = None
        self.output = None
        # er wordt een random getal tussen 0 en 1 gemaakt bij de weights en biases
        # voor de weights wordt er een array gemaakt met het 'input_data' aantal aan data 
        # en 'outpout'het aantal getallen elk rij heeft en daarna wordt een half ervan afgehaald
        self.weights = np.random.rand(input_data, output_data) - 0.5
        # voor de bias wordt er een rij gemaakt met 5 random values en van elke value wordt daarna een halve afgehaald
        self.bias = np.random.rand(1, output_data) - 0.5
        self.activation = activation
        self.der_activation = der_activation
    
    # berekend het output van een laag
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(np.dot(self.input, self.weights) + self.bias)
        return self.output
    
    # berekend de backprogation en verbeterd hierbij ook de weights en biases aan de hand van de learning rate
    def backward_propagation(self, output_error, learning_rate):
        input_error = self.der_activation(self.output) * output_error
        input_error = np.dot(input_error, self.weights.T)
        
        weights_error = np.dot(self.input.T, output_error)
        
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        
        return input_error

## Network samenstellen

In [67]:
class Network:
    #netwerk initialiseren
    def __init__(self):
        self.layers = []
        self.cost = None
        self.der_cost = None 
    
    # een functie voor het toevoegen van de laag
    def add(self, layer):
        self.layers.append(layer)
    
    # een functie waarbij de kostfunctie en de afgeleide ervan wordt meegegeven
    def use(self, cost, der_cost):
        self.cost = cost
        self.der_cost = der_cost
    
    # deze predict functie wordt gebtuikt voor het uitvoeren van de nn
    def predict(self, input_data):
        # om de dimensie/lengte van de de input data
        samples = len(input_data)
        # leeg array die alle resultaten bewaart
        result = []
        
        # er wordt gelooped aan de hand van elke lengte van de data
        for i in range(samples):
            # de output wordt hier al geinitialiseerd en wordt het inputdata
            output = input_data[i]
            # voor elke laag in het network wordt de input gestuurd naar de forward propegation van de laag en wordt de return 
            # toegevoegd aan de array
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result
    
    #de fit functie is voor het trainen van het netwerk
    def fit(self, x_train, y_train, epochs, learning_rate):
        samples = len(x_train)
        
        #een epoch is wanneer de data voor het trainen een keer helemaal door het netwerk is gegaan
        for i in range(epochs):
            err = 0
            
            #net zoals bij de predict methode wordt aan de hand van de lengte elke data naar de feedforward gestuurd
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                
                # aan de hand van de output wordt de resultaat met het gewilde antwoord vergeleken
                err += self.cost(y_train[j], output)
                
                # aan de hand van de afgeleide van de mse weten de richting waarin we de parameters aanpassen
                error = self.der_cost(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
            
            # het gemiddelde van alle trainingsdatta
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

## Een kleine test

In [68]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

In [69]:
net = Network()
net.add(Layer(2, 3, sig, der_sig))
net.add(Layer(3, 1, relu, der_relu))

In [70]:
net.use(mse, der_mse)
net.fit(x_train, y_train, epochs=10, learning_rate=0.1)

epoch 1/10   error=0.515989
epoch 2/10   error=0.376097
epoch 3/10   error=0.346439
epoch 4/10   error=0.346506
epoch 5/10   error=0.346270
epoch 6/10   error=0.345771
epoch 7/10   error=0.345229
epoch 8/10   error=0.344692
epoch 9/10   error=0.344172
epoch 10/10   error=0.343668


## Testen met een grotere datasets

In [71]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [72]:
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255

In [73]:
y_train = np_utils.to_categorical(y_train)

In [74]:
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

In [75]:
net = Network()
net.add(Layer(28*28, 100, sig, der_sig))
net.add(Layer(100, 50, sig, der_sig)) 
net.add(Layer(50, 10, sig, der_sig))

In [76]:
net.use(mse, der_mse)
net.fit(x_train[0:1000], y_train[0:1000], epochs=50, learning_rate=0.1)

epoch 1/50   error=0.079482
epoch 2/50   error=0.050860
epoch 3/50   error=0.038651
epoch 4/50   error=0.031427
epoch 5/50   error=0.026657
epoch 6/50   error=0.023501
epoch 7/50   error=0.021347
epoch 8/50   error=0.019782
epoch 9/50   error=0.018550
epoch 10/50   error=0.017577
epoch 11/50   error=0.016845
epoch 12/50   error=0.016290
epoch 13/50   error=0.015811
epoch 14/50   error=0.015319
epoch 15/50   error=0.014772
epoch 16/50   error=0.014176
epoch 17/50   error=0.013563
epoch 18/50   error=0.012962
epoch 19/50   error=0.012360
epoch 20/50   error=0.011751
epoch 21/50   error=0.011185
epoch 22/50   error=0.010681
epoch 23/50   error=0.010218
epoch 24/50   error=0.009829
epoch 25/50   error=0.009550
epoch 26/50   error=0.009418
epoch 27/50   error=0.009433
epoch 28/50   error=0.009440
epoch 29/50   error=0.009385
epoch 30/50   error=0.009271
epoch 31/50   error=0.009119
epoch 32/50   error=0.009028
epoch 33/50   error=0.008966
epoch 34/50   error=0.008891
epoch 35/50   error=0.0

In [77]:
for i in range(len(x_test)):
    out = net.predict(x_test[i])
    print("predicted values - true values : " )
    print(out)
    print(" - ")
    print(y_test[i])
    print("\n")

predicted values - true values : 
[array([[3.88538404e-03, 3.18313761e-06, 1.62427264e-02, 8.00098905e-04,
        2.27582388e-05, 2.49461834e-05, 4.61611674e-06, 9.98259419e-01,
        1.47839016e-05, 4.26201618e-02]])]
 - 
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


predicted values - true values : 
[array([[4.67080041e-02, 1.59792454e-05, 9.54141495e-02, 3.63364807e-01,
        1.74583250e-06, 8.42976352e-03, 6.81628988e-02, 1.74432907e-05,
        1.90741683e-06, 1.04663503e-03]])]
 - 
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[2.82859319e-07, 9.68634748e-01, 1.30730111e-02, 1.01976384e-02,
        3.18841120e-05, 2.18864516e-06, 2.19730660e-04, 6.41784163e-02,
        1.04937136e-04, 9.73155433e-05]])]
 - 
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[8.94075695e-01, 1.13253282e-05, 1.25614831e-03, 3.40802350e-05,
        1.96503669e-05, 3.71743807e-04, 1.65610800e-02, 2.60482162e-01,
        4.36880051e-08, 4.00183370e-04]])

predicted values - true values : 
[array([[1.84641119e-04, 1.70792688e-04, 1.58804909e-04, 6.52574595e-06,
        9.18039574e-01, 1.89880577e-06, 1.33382561e-03, 6.32419892e-02,
        3.36834850e-07, 9.14539513e-03]])]
 - 
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[2.51565997e-03, 2.75593248e-05, 6.97370937e-04, 4.96741939e-05,
        4.76780268e-05, 5.30338889e-03, 8.98835986e-01, 1.69887608e-06,
        1.27941109e-06, 2.10116003e-02]])]
 - 
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


predicted values - true values : 
[array([[3.97890985e-04, 1.45192326e-04, 1.64568686e-04, 9.94258334e-01,
        6.31074910e-05, 1.17420199e-01, 1.99205470e-04, 1.31869622e-04,
        8.06960385e-06, 6.63794606e-06]])]
 - 
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[5.47447956e-03, 1.29357961e-04, 1.63374825e-09, 5.80861631e-05,
        5.62790634e-01, 7.30521432e-01, 1.21543571e-05, 5.45240327e-04,
        2.06019025e-08, 4.26505980e-05]])

predicted values - true values : 
[array([[2.33243393e-05, 2.63525221e-04, 5.96862186e-05, 1.73831097e-03,
        9.74024434e-01, 7.04979508e-06, 5.50765953e-01, 7.10680838e-06,
        6.18654820e-09, 2.05933199e-03]])]
 - 
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[3.00621159e-07, 5.28881369e-04, 1.29253635e-03, 1.35366104e-06,
        9.80846614e-03, 6.00818764e-03, 9.96621683e-01, 1.33262251e-05,
        2.83378196e-05, 1.01606437e-05]])]
 - 
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


predicted values - true values : 
[array([[4.03792216e-02, 1.24289170e-05, 8.20044770e-01, 6.21178052e-04,
        3.71619821e-04, 1.06173083e-03, 1.53471700e-06, 1.77253260e-01,
        2.72868585e-05, 3.60954640e-05]])]
 - 
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[3.96685072e-08, 5.20205236e-01, 9.23699233e-01, 1.33883692e-02,
        8.41687686e-05, 8.73065637e-07, 7.32182889e-02, 4.05740383e-04,
        6.26948227e-05, 1.20706956e-03]])

predicted values - true values : 
[array([[1.88207092e-08, 6.02608944e-02, 5.51325394e-06, 9.51114045e-01,
        3.76487296e-04, 4.99722265e-03, 2.27314203e-05, 1.54386256e-03,
        3.43693207e-04, 3.00487904e-04]])]
 - 
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[2.12776928e-07, 5.48069012e-01, 5.74880992e-01, 6.72104098e-04,
        3.23322667e-06, 2.78800891e-03, 8.23900640e-03, 7.82713141e-05,
        2.52060523e-02, 4.23375554e-06]])]
 - 
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[5.06973500e-08, 2.26694180e-03, 3.13094190e-09, 3.91919572e-01,
        2.46988040e-03, 2.69889700e-03, 5.51392205e-06, 1.36006039e-02,
        4.76012746e-06, 7.03854382e-01]])]
 - 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


predicted values - true values : 
[array([[1.35767083e-05, 2.53998175e-04, 6.59841824e-03, 8.06177303e-06,
        1.09519439e-04, 8.34635783e-04, 6.37301040e-05, 9.92574318e-01,
        5.13375571e-04, 5.56814853e-05]])

predicted values - true values : 
[array([[1.86784489e-07, 9.92231891e-01, 1.42459153e-04, 3.79902777e-03,
        1.98348046e-06, 2.45208563e-05, 8.37819469e-04, 1.52119797e-02,
        2.63514027e-03, 2.12686812e-04]])]
 - 
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[5.58973215e-05, 1.45929194e-05, 7.38852768e-07, 4.41657638e-03,
        9.94270549e-01, 4.65271105e-04, 4.76639584e-04, 7.75151948e-05,
        4.35901533e-05, 1.64085640e-02]])]
 - 
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[2.54503127e-06, 1.47585882e-05, 1.18408734e-06, 1.06892895e-01,
        9.82451863e-01, 7.90780757e-04, 4.55562317e-04, 7.74889135e-05,
        3.86453836e-05, 2.66481940e-02]])]
 - 
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[1.64505826e-05, 9.64847292e-04, 1.84516355e-08, 9.82252956e-01,
        9.50640178e-05, 2.46293761e-01, 1.33952243e-05, 4.70255183e-05,
        6.51482810e-07, 9.43269913e-02]])

...

predicted values - true values : 
[array([[1.70886138e-03, 5.51780078e-04, 5.20736994e-01, 1.86929215e-02,
        1.04515576e-03, 6.27040856e-06, 1.27184022e-05, 1.04875180e-03,
        4.57454419e-06, 4.77899600e-02]])]
 - 
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[1.49169004e-05, 1.19284804e-04, 3.96011880e-03, 7.18254633e-06,
        1.72760082e-02, 2.24110180e-04, 9.84911061e-01, 1.11053641e-06,
        7.19324014e-06, 1.42635647e-02]])]
 - 
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


predicted values - true values : 
[array([[2.83226311e-03, 2.33954401e-04, 2.45507688e-01, 7.99716465e-01,
        5.23312684e-04, 1.46300236e-04, 4.62081936e-06, 1.08687944e-03,
        3.14311330e-05, 8.56285271e-04]])]
 - 
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[2.02124211e-05, 5.39983888e-03, 2.90069727e-04, 6.33697577e-06,
        9.34331635e-04, 3.63986074e-02, 1.11843081e-05, 1.09822339e-03,
        9.92933715e-01, 1.13002923e-03]])

predicted values - true values : 
[array([[3.12093386e-06, 3.47417297e-03, 5.16691245e-02, 9.57722257e-01,
        3.40826642e-03, 5.09776421e-05, 4.29236963e-06, 3.80964628e-04,
        7.80717414e-04, 1.86551936e-03]])]
 - 
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[4.00476271e-04, 7.69792658e-06, 7.21940283e-09, 3.80728573e-04,
        9.38653863e-04, 3.69660277e-02, 4.74802400e-06, 6.31104193e-03,
        1.81341106e-05, 8.70328859e-01]])]
 - 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


predicted values - true values : 
[array([[6.24349517e-02, 7.61771879e-05, 2.32878177e-03, 5.59752863e-06,
        1.48076251e-03, 9.16568527e-04, 9.61994157e-01, 7.24544072e-07,
        4.22098518e-08, 2.33277167e-03]])]
 - 
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


predicted values - true values : 
[array([[4.03097996e-07, 9.93297374e-01, 1.01000250e-02, 2.07964202e-03,
        1.29994676e-04, 1.66814565e-05, 9.66771103e-05, 3.24548080e-02,
        6.55476074e-04, 1.99076381e-06]])

predicted values - true values : 
[array([[2.18873237e-04, 1.68794399e-05, 1.44359789e-06, 2.27149664e-05,
        6.20781063e-05, 1.84463731e-03, 2.87085549e-07, 9.99125134e-01,
        9.66718595e-06, 3.81361436e-02]])]
 - 
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


predicted values - true values : 
[array([[3.84577938e-06, 6.66665405e-03, 8.60985762e-03, 1.51736532e-03,
        2.36075204e-05, 4.41976929e-02, 4.15682311e-05, 6.53525337e-06,
        8.41746348e-01, 2.53013401e-03]])]
 - 
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


predicted values - true values : 
[array([[3.61377466e-07, 8.38785208e-03, 1.03450167e-03, 6.69305728e-06,
        3.62098898e-03, 8.30016958e-07, 3.96475059e-03, 1.85858026e-02,
        3.89368642e-04, 8.98152736e-01]])]
 - 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


predicted values - true values : 
[array([[5.70875349e-01, 3.90735993e-05, 8.13292707e-02, 2.79741316e-07,
        7.86805395e-06, 1.62368515e-04, 1.22410969e-02, 2.50032244e-02,
        1.05302565e-07, 4.70355641e-03]])

predicted values - true values : 
[array([[9.89679890e-01, 8.73595695e-06, 6.55014360e-04, 3.12224383e-03,
        8.07353785e-06, 1.59199176e-02, 4.21036013e-04, 2.82000235e-04,
        1.11668410e-07, 4.53292420e-03]])]
 - 
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[9.92326179e-01, 1.56859094e-05, 4.60987555e-04, 1.25101318e-02,
        9.43497543e-06, 9.03230359e-02, 1.18096436e-05, 4.24178232e-04,
        1.45396713e-07, 5.50052604e-04]])]
 - 
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[2.95605128e-07, 3.24423041e-01, 2.13730819e-03, 4.76231670e-05,
        3.03551703e-03, 1.44068896e-04, 8.09380834e-04, 1.48115241e-02,
        1.63404710e-01, 1.66605164e-05]])]
 - 
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[2.97083592e-02, 7.87334613e-06, 1.87191907e-06, 3.71092064e-05,
        3.17891241e-04, 4.15616723e-03, 4.49586823e-06, 9.97245670e-01,
        3.12613158e-07, 4.20667397e-04]])

predicted values - true values : 
[array([[2.35223274e-07, 9.90527777e-01, 1.66459069e-02, 1.01862505e-02,
        1.03596070e-03, 7.12113706e-06, 5.94909337e-05, 9.60575372e-03,
        3.82299798e-04, 2.35934970e-06]])]
 - 
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[8.81960528e-03, 1.40715587e-04, 2.05764869e-03, 1.27579492e-05,
        2.91419677e-05, 3.75397411e-04, 2.26022408e-02, 3.20832089e-05,
        4.48650841e-05, 1.99971025e-01]])]
 - 
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[3.82343311e-06, 3.63620574e-02, 9.64017775e-01, 2.15001822e-02,
        1.15659432e-05, 4.17827682e-04, 1.36686666e-04, 9.68642122e-05,
        1.34888612e-01, 1.75315213e-04]])]
 - 
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


predicted values - true values : 
[array([[4.27860105e-03, 6.78467032e-03, 7.22302527e-07, 3.00718434e-01,
        1.60797585e-06, 4.50897895e-01, 1.67776854e-04, 1.81746006e-04,
        2.81437930e-07, 5.50990034e-06]])